In [1]:
import pandas as pd
import os
import math
import json
import glob
import collections
import random
from pathlib import Path
import pandas as pd
import numpy as np
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import copy
from tqdm.auto import tqdm
import pickle
import gc
from sklearn.model_selection import StratifiedKFold,KFold,GroupKFold
import torch
# pip install prefetch_generator
from prefetch_generator import BackgroundGenerator
os.environ["TOKENIZERS_PARALLELISM"] = "true"
from  transformers import AdamW, AutoTokenizer,AutoModel
import torch.nn as nn
import torch.nn.functional as F
import warnings
import torch.nn as nn
import torch
from transformers import AutoModel, AutoTokenizer
from functools import partial
from torch.utils.data import DataLoader
dataloader_class = partial(DataLoader, pin_memory=True, num_workers=32)

/root/panpingjun/Anaconda/chatglm_ppj/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
target_articles = ['API gravity', 'Amplitude', 'Angular momentum', 'Antiferromagnetism', 'Astrochemistry', 'Baryogenesis', 'Black hole', 'Bollard pull', 'Born reciprocity', 'Butterfly effect', 'C1 chemistry', 'Causality (physics)', 'Cavitation', 'Clockwise', 'Coffee ring effect', 'Coherence (physics)', 'Coherent turbulent structure', 'Cold dark matter', "Commentary on Anatomy in Avicenna's Canon", 'Condensation cloud', 'Convection (heat transfer)', 'Copernican principle', 'Critical Raw Materials Act', 'Crossover experiment (chemistry)', 'Crystallinity', 'Dark Matter', 'Decay technique', 'Diffraction', 'Dimension', 'Droste effect', 'Dynamic scaling', "Earnshaw's theorem", 'Ecological pyramid', 'Electric flux', 'Electrical resistivity and conductivity', 'Electrochemical gradient', 'Electronic entropy', "Elitzur's theorem", 'Emissivity', 'Enthalpy', 'Environmental Science Center', 'Erlangen program', 'Explicit symmetry breaking', "Fermat's principle", 'Ferromagnetism', 'Frame-dragging', 'Free neutron decay', 'Galaxy', 'Geometric quantization', 'Gravitational wave', 'Gravity Probe B', 'Heart', 'Heat treating', "Hesse's principle of transfer", 'History of geology', 'Hydrodynamic stability', 'Improper rotation', 'Infectious tolerance', 'Inflation (cosmology)', 'Interstellar medium', 'James Webb Space Telescope', 'Kutta-Joukowski theorem', 'Landau–Lifshitz–Gilbert equation', 'Leidenfrost effect', 'Light-year', 'Linear time-invariant system', 'List of equations in classical mechanics', 'Lorentz covariance', 'Luminance', 'Magnetic monopole', 'Magnetic resonance imaging', 'Magnetic susceptibility', 'Magnitude (astronomy)', 'Main sequence', 'Mammary gland', 'Mass versus weight', 'Mass-to-charge ratio', 'Memristor', 'Minkowski space', 'Modified Newtonian dynamics', 'Molecular cloud', 'Molecular symmetry', 'Morphology (biology)', 'Navier–Stokes equations', 'Nebula', "Newton's law of universal gravitation", 'Nuclear fusion', 'Observable universe', 'Organography', 'Paramagnetism', 'Parity (physics)', 'Phageome', 'Phase transition', 'Photophoresis', 'Planetary system', 'Plant', 'Point groups in three dimensions', 'Probability amplitude', 'Probability density function', 'Propagation constant', 'Pulsar', 'Pulsar-based navigation', 'QCD matter', 'Quantization (physics)', 'Quartz crystal microbalance', 'Radiosity (radiometry)', 'Ramsauer–Townsend effect', 'Rayleigh scattering', 'Reciprocal length', 'Redshift', 'Refractive index', 'Regular polytope', 'Relative density', 'Renormalization', 'Ring-imaging Cherenkov detector', 'Scale (ratio)', 'Second law of thermodynamics', 'Self-organization in cybernetics', 'Shower-curtain effect', 'Signal-to-noise ratio', 'Spatial dispersion', 'Speed of light', 'Spin (physics)', 'Spontaneous symmetry breaking', 'Standard Model', 'Stellar classification', 'Stochastic differential equation', 'Superconductivity', 'Supermassive black hole', 'Supernova', 'Supersymmetric quantum mechanics', 'Supersymmetry', 'Surface power density', 'Surgical pathology', 'Symmetry in biology', 'Symmetry of diatomic molecules', 'The Ambidextrous Universe', 'Theorem', 'Theorem of three moments', 'Thermal equilibrium', 'Tidal force', 'Time', 'Time standard', 'Total internal reflection', 'Triskelion', 'Ultraviolet catastrophe', 'Unified field theory', 'Uniform tilings in hyperbolic plane', 'Vacuum', 'Virtual particle', 'Water hammer', 'Wigner quasiprobability distribution', 'Work function', 'Zero-point energy']

In [3]:
data = []
listdir = os.listdir('./wiki_data')
listdir = sorted(listdir)
for file in listdir:
    if file == 'my_index.parquet' or 'ipynb' in file or '680w' in file or 'kmean' in file:
        continue
    print(file)
    data.append(pd.read_parquet(os.path.join('./wiki_data',file)))

a.parquet
b.parquet
c.parquet
d.parquet
e.parquet
f.parquet
g.parquet
h.parquet
i.parquet
j.parquet
k.parquet
l.parquet
m.parquet
n.parquet
number.parquet
o.parquet
other.parquet
p.parquet
q.parquet
r.parquet
s.parquet
t.parquet
u.parquet
v.parquet
w.parquet
x.parquet
y.parquet
z.parquet


In [4]:
data = pd.concat(data,axis=0).reset_index(drop=True)

In [5]:
data['concat'] = data['title'] + ' ' + data['text']

In [6]:
from tqdm.auto import tqdm
import torch
class LLMRecallDataSet(torch.utils.data.Dataset):
    def __init__(self, data):
        self.tokenizer = AutoTokenizer.from_pretrained('/root/bert_path/sentence-transformer-all-mpnet-base-v2', use_fast=True)
        self.data = data
    def __len__(self):
        return len(self.data) 
    
    def __getitem__(self,index):
        ids = self.data.loc[index, 'title'] + ' ' + self.data.loc[index,'text']
        ids = self.tokenizer.encode(ids, add_special_tokens=False)
        if len(ids) > 510:
            ids = [101] + ids[:510] + [102]
        else:
            ids = [101] + ids + [102]
        return ids
    def collate_fn(self, batch):
        def sequence_padding(inputs, length=None, padding=0):
            """
            Numpy函数，将序列padding到同一长度
            """
            if length is None:
                length = max([len(x) for x in inputs])

            pad_width = [(0, 0) for _ in np.shape(inputs[0])]
            outputs = []
            for x in inputs:
                x = x[:length]
                pad_width[0] = (0, length - len(x))
                x = np.pad(x, pad_width, 'constant', constant_values=padding)
                outputs.append(x)

            return np.array(outputs, dtype='int64')
        batch_ids = torch.tensor(sequence_padding(batch), dtype=torch.long)
        
        return batch_ids

        
class DataLoaderX(torch.utils.data.DataLoader):
    '''
        replace DataLoader with PrefetchDataLoader
    '''
    def __iter__(self):
        return BackgroundGenerator(super().__iter__())  

    
def get_loader(prompt,batch_size,train_mode=True,num_workers=4):
    ds_df = LLMRecallDataSet(prompt)
    # loader = DataLoaderX(ds_df, batch_size=batch_size if train_mode else batch_size//2, shuffle=train_mode, num_workers=num_workers,pin_memory=True,
    #                                      collate_fn=ds_df.collate_fn, drop_last=train_mode)
    loader = dataloader_class(ds_df, batch_size=batch_size, shuffle=False,collate_fn=ds_df.collate_fn,num_workers=num_workers)
    loader.num = len(ds_df)
    return loader

In [1]:
model = AutoModel.from_pretrained('/root/bert_path/sentence-transformer-all-mpnet-base-v2')
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
model.to('cuda')
model = torch.nn.parallel.DataParallel(model)
loader = get_loader(data, 8000, False, num_workers=16)
from torch.cuda.amp import autocast
alls = []
model.eval()
idx = 0
with torch.no_grad():
    for batch in tqdm(loader):
        ids = batch
        ids = ids.to('cuda')
        with autocast():
            output = model(input_ids=ids)
        attention_mask = ids > 0
        output = mean_pooling(output,attention_mask).cpu().detach().numpy()
        alls.append(output)
alls = np.concatenate(alls, axis=0)

In [7]:
import torch.nn.functional as F
alls = F.normalize(torch.from_numpy(alls), p=2, dim=1).numpy()

In [6]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('/root/bert_path/sentence-transformer-all-mpnet-base-v2', device='cuda')
model.max_seq_length = 512
model = model.half()
## Get embeddings of the wiki text data
alls = model.encode(data.concat,
                    batch_size=1000,
                    device='cuda',
                    show_progress_bar=True,
                    convert_to_tensor=True,
                    normalize_embeddings=True)#.half()
alls = alls.detach().cpu().numpy()

[2023-10-10 18:30:25,765] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


KeyboardInterrupt: 

In [10]:
np.save('./tmp/680w_embed.npy', alls)

In [7]:
wiki_embed = np.load('./tmp/680w_embed.npy')

In [8]:
from sklearn.cluster import KMeans
import numpy as np

kmeans = KMeans(n_clusters=35,random_state=0,n_init=2, n_jobs=-1).fit(wiki_embed)

/root/panpingjun/Anaconda/chatglm_ppj/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:792: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


In [9]:
cluster_numbers = kmeans.predict(wiki_embed)

In [10]:
cluster_numbers.shape

(6286775,)

In [11]:
import collections

cluster_dict = collections.defaultdict(list) # membership dictionary for each cluster
for dx,cl_id in tqdm(enumerate(cluster_numbers),total=cluster_numbers.shape[0]):
    cluster_dict[cl_id].append(data.loc[dx,'title'])

100%|██████████| 6286775/6286775 [00:46<00:00, 135468.43it/s]


In [12]:
sorted([(cluster_numbers[dx],i) for dx,i in tqdm(enumerate(data['title'].tolist())) if i in target_articles])

6286775it [00:13, 470075.73it/s]


[(6, 'Organography'),
 (8, 'API gravity'),
 (8, 'Bollard pull'),
 (8, 'Cavitation'),
 (8, 'Clockwise'),
 (8, 'Critical Raw Materials Act'),
 (8, 'Electric flux'),
 (8, 'Electronic entropy'),
 (8, "Elitzur's theorem"),
 (8, 'Emissivity'),
 (8, 'Galaxy'),
 (8, 'Heat treating'),
 (8, 'James Webb Space Telescope'),
 (8, 'Linear time-invariant system'),
 (8, 'Luminance'),
 (8, 'Mass versus weight'),
 (8, 'Memristor'),
 (8, 'Minkowski space'),
 (8, 'Observable universe'),
 (8, 'Propagation constant'),
 (8, 'Pulsar-based navigation'),
 (8, 'Regular polytope'),
 (8, 'Signal-to-noise ratio'),
 (8, 'Spatial dispersion'),
 (8, 'Stochastic differential equation'),
 (8, 'Surface power density'),
 (8, 'Theorem of three moments'),
 (8, 'Thermal equilibrium'),
 (8, 'Time'),
 (8, 'Time standard'),
 (8, 'Uniform tilings in hyperbolic plane'),
 (8, 'Water hammer'),
 (8, 'Work function'),
 (13, 'Amplitude'),
 (13, 'Angular momentum'),
 (13, 'Antiferromagnetism'),
 (13, 'Astrochemistry'),
 (13, 'Baryogenes

In [13]:
filtered_articles = cluster_dict[13]
filtered_articles_set = set(filtered_articles)
len(filtered_articles_set)

148313

In [21]:
filtered_articles1 = cluster_dict[8]
filtered_articles_set1 = set(filtered_articles1)
len(filtered_articles_set1)

259724

In [22]:
all_set = filtered_articles_set | filtered_articles_set1

In [23]:
filte_data = data[data['title'].isin(all_set)]

In [24]:
filte_data = filte_data.reset_index(drop=True)

In [25]:
filte_data

,id,title,text,categories
0,15547032,A & G Price,A & G Price Limited is an engineering firm and...,"[Locomotive manufacturers of New Zealand, Tham..."
1,73863747,A (For 100 Cars),A [For 100 Cars] is a minimalist composition b...,"[2017 compositions, Minimalistic compositions,..."
2,5405379,A (S-train),A is a service on the S-train network in Copen...,[S-train (Copenhagen)]
3,2828410,A (musical note),A or La is the sixth note and the tenth semito...,[Musical notes]
4,48227028,A B M Shawkat Ali,A B M Shawkat Ali is a Bangladeshi origin-Aust...,"[Living people, 1969 births, University of Raj..."
...,...,...,...,...
408032,36642977,Zündwaren monopoly,"thumb|200px|Post-war Welthölzer, Deutsche Zünd...","[Monopoly (economics), 1930 in Germany, Econom..."
408033,988185,Zürcher Verkehrsverbund,"The Zürcher Verkehrsverbund (ZVV, Zürich Trans...","[Public transport in Switzerland, Transport in..."
408034,10709120,Zürich Metropolitan Area,thumb|Location within Switzerland The European...,"[Geography of the canton of Zürich, Metropolit..."
408035,44312982,Zürichsee-Zeitung,"Zürichsee-Zeitung, commonly shortened to ZSZ, ...","[1845 establishments in Switzerland, Daily new..."


In [12]:
filte_data.to_parquet('./wiki_data/kmeans_data.parquet')

NameError: name 'filte_data' is not defined

In [13]:
filte_data = pd.read_parquet('./wiki_data/kmeans_data.parquet')

In [92]:
data_base = pd.read_parquet('./small_wiki_data_base/data.parquet')

In [93]:
have_id = set(data_base['wiki_id'].apply(lambda x : int(x)).tolist())

In [94]:
now_id = set(filte_data['id'].apply(lambda x : int(x)).tolist())

In [95]:
need_id = (now_id - (now_id & have_id))

In [96]:
len(need_id)

114084

In [97]:
filte_data = filte_data[filte_data['id'].apply(lambda x : int(x)).isin(need_id)].reset_index(drop=True)

In [14]:
from __future__ import annotations

import blingfire as bf
from typing import Optional, Union
from collections.abc import Iterable

def process_documents(documents: Iterable[str],
                      document_ids: Iterable,
                      split_sentences: bool = True,
                      filter_len: int = 3,
                      disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Main helper function to process documents from the EMR.

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param document_type: String denoting the document type to be processed
    :param document_sections: List of sections for a given document type to process
    :param split_sentences: Flag to determine whether to further split sections into sentences
    :param filter_len: Minimum character length of a sentence (otherwise filter out)
    :param disable_progress_bar: Flag to disable tqdm progress bar
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `section`, `offset`
    """
    
    df = sectionize_documents(documents, document_ids, disable_progress_bar)

    if split_sentences:
        df = sentencize(df.text.values, 
                        df.document_id.values,
                        df.offset.values, 
                        filter_len, 
                        disable_progress_bar)
    return df


def sectionize_documents(documents: Iterable[str],
                         document_ids: Iterable,
                         disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Obtains the sections of the imaging reports and returns only the 
    selected sections (defaults to FINDINGS, IMPRESSION, and ADDENDUM).

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param disable_progress_bar: Flag to disable tqdm progress bar
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `offset`
    """
    processed_documents = []
    for document_id, document in tqdm(zip(document_ids, documents), total=len(documents), disable=disable_progress_bar):
        row = {}
        text, start, end = (document, 0, len(document))
        row['document_id'] = document_id
        row['text'] = text
        row['offset'] = (start, end)

        processed_documents.append(row)

    _df = pd.DataFrame(processed_documents)
    if _df.shape[0] > 0:
        return _df.sort_values(['document_id', 'offset']).reset_index(drop=True)
    else:
        return _df


def sentencize(documents: Iterable[str],
               document_ids: Iterable,
               offsets: Iterable[tuple[int, int]],
               filter_len: int = 3,
               disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Split a document into sentences. Can be used with `sectionize_documents`
    to further split documents into more manageable pieces. Takes in offsets
    to ensure that after splitting, the sentences can be matched to the
    location in the original documents.

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param offsets: Iterable tuple of the start and end indices
    :param filter_len: Minimum character length of a sentence (otherwise filter out)
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `section`, `offset`
    """

    document_sentences = []
    for document, document_id, offset in tqdm(zip(documents, document_ids, offsets), total=len(documents), disable=disable_progress_bar):
        try:
            _, sentence_offsets = bf.text_to_sentences_and_offsets(document)
            for o in sentence_offsets:
                if o[1]-o[0] > filter_len:
                    sentence = document[o[0]:o[1]]
                    abs_offsets = (o[0]+offset[0], o[1]+offset[0])
                    row = {}
                    row['document_id'] = document_id
                    row['text'] = sentence
                    row['offset'] = abs_offsets
                    document_sentences.append(row)
        except:
            continue
    return pd.DataFrame(document_sentences)

In [15]:
processed = process_documents(filte_data.text, filte_data.id)

100%|██████████| 270492/270492 [11:09<00:00, 404.11it/s]


In [100]:
processed

,document_id,text,offset
0,10000242,"Rod J. Rohrich ( ), F.A.C.S. is a Dallas-based...","(0, 84)"
1,10000242,He is the editor-in-chief of the journal Plast...,"(85, 266)"
2,10000242,He is board certified by the American Board of...,"(267, 330)"
3,10000242,A June 2014 journal article in Annals of Plast...,"(331, 622)"
4,10000242,"In 2021, Newsweek recognized Dr. Rohrich as th...","(623, 837)"
...,...,...,...
3694546,9999956,Evol.,"(579, 584)"
3694547,9999956,"Microbiol., 50, 1811-1816. ==Physiology== M. k...","(585, 716)"
3694548,9999956,Isolates have been shown to be resistant to th...,"(717, 904)"
3694549,9999956,"Susceptibility to clarithromycin, clofazimine ...","(905, 1094)"


In [101]:
d = {}
for i in range(len(data)):
    d[int(data.loc[i,'id'])] = i

In [102]:
new_data = []
# for page_id in tqdm(processed['document_id'].unique()):
#     sub = processed[processed['document_id'] == page_id].reset_index(drop=True)
#     text = ''
#     title = data.loc[d[int(page_id)],'title']
#     for item in sub['text'].to_list():
#         text += item
#         if len(text) > 2500:
#             new_data.append({'id':int(page_id), 'title':title, 'text':text})
#             text = ""
#     if len(text) != 0:
#         new_data.append({'id':int(page_id), 'title':title, 'text':text})
page_ids = set(processed['document_id'].tolist())
texts = {}
for page_id in page_ids:
    texts[page_id] = []
for i in tqdm(range(len(processed))):
    texts[processed.loc[i,'document_id']].append(processed.loc[i,'text'])

100%|██████████| 3694551/3694551 [00:52<00:00, 70112.15it/s]


In [111]:
import re

new_data = []
for page_id in tqdm(page_ids):
    text = ''
    title = data.loc[d[int(page_id)], 'title']
    for item in texts[page_id]:
        item = re.sub(r'http\S+', '', item)
        item = re.sub(r'[+\-*\/]', '', item)
        item = re.sub(r'\$\\frac\{.*?\}\{.*?\}\$', '', item)
        if len(text + item) > 2500:
            new_data.append({'id':int(page_id), 'title':title, 'text':text})
            text = item
            continue
        text += item
    if len(text) != 0:
        new_data.append({'id':int(page_id), 'title':title, 'text':text})

100%|██████████| 114084/114084 [00:19<00:00, 5717.96it/s]


In [112]:
new_data = pd.DataFrame(new_data)

In [113]:
new_data['text_len'] = new_data['text'].apply(lambda x : len(x))

In [114]:
new_data = new_data[new_data['text_len'] != 0].reset_index(drop=True)
new_data = new_data[['id','title', 'text']]

In [115]:
new_data

,id,title,text
0,893962,UC Davis School of Veterinary Medicine,"The University of California, Davis, School of..."
1,893962,UC Davis School of Veterinary Medicine,"The tracks are Equine Track, EquineSmall Anima..."
2,893962,UC Davis School of Veterinary Medicine,Five faculty members (Terrell Holliday [1998]...
3,893962,UC Davis School of Veterinary Medicine,Notable discoveries by faculty of the School ...
4,22185825,Ancient Egyptian creation myths,Ancient Egyptian creation myths are the ancien...
...,...,...,...
335234,29080755,List of lunar probes,Then sent to L2 and on to an asteroid flyby.AR...
335235,29080755,List of lunar probes,Chandrayaan2 ISRO 22 July 2019 – orbiter in or...
335236,29080755,List of lunar probes,Communication failure. 50x50px ArgoMoon ASI 21...
335237,29080755,List of lunar probes,Lunar Flashlight 11 December 2022 (launch) orb...


In [116]:
new_data.to_parquet('./small_wiki_data_base/680w_kmeans_split_2500_k35_clear.parquet')

In [117]:
merge = pd.concat([data_base, new_data],axis=0).reset_index(drop=True)
merge['id'] = merge['id'].apply(lambda x : str(x))
merge.to_parquet('./small_wiki_data_base/merge_k35_2500_clear.parquet')

In [118]:
merge

,id,title,text,url,wiki_id,views,paragraph_id,langs
0,5443,Google Translate,Google Translate is a multilingual neural mach...,https://en.wikipedia.org/wiki?curid=3235536,3235536.0,4305.652832,0.0,97.0
1,5444,Google Translate,Launched in April 2006 as a statistical machin...,https://en.wikipedia.org/wiki?curid=3235536,3235536.0,4305.652832,1.0,97.0
2,5445,Google Translate,Google Translate is a web-based free-to-user t...,https://en.wikipedia.org/wiki?curid=3235536,3235536.0,4305.652832,2.0,97.0
3,5446,Google Translate,"Originally, Google Translate was released as a...",https://en.wikipedia.org/wiki?curid=3235536,3235536.0,4305.652832,3.0,97.0
4,5447,Google Translate,"In January 2010, Google introduced an Android ...",https://en.wikipedia.org/wiki?curid=3235536,3235536.0,4305.652832,4.0,97.0
...,...,...,...,...,...,...,...,...
3116886,29080755,List of lunar probes,Then sent to L2 and on to an asteroid flyby.AR...,NaN,NaN,NaN,NaN,NaN
3116887,29080755,List of lunar probes,Chandrayaan2 ISRO 22 July 2019 – orbiter in or...,NaN,NaN,NaN,NaN,NaN
3116888,29080755,List of lunar probes,Communication failure. 50x50px ArgoMoon ASI 21...,NaN,NaN,NaN,NaN,NaN
3116889,29080755,List of lunar probes,Lunar Flashlight 11 December 2022 (launch) orb...,NaN,NaN,NaN,NaN,NaN
